##RTI Data Challenge
This notebook is in reference to the following challenge: https://github.com/rtidatascience/exercises/tree/master/exercise01

###Task 1:
**Write a SQL query that creates a consolidated dataset from the normalized tables in the database. In other words, write a SQL query that "flattens" the database to a single table.**

First things first, let's initialize our connection to the SQLite DB we were provided with.

In [18]:
#Initialize Connection to SQLite Database
import sqlite3
sqlite_file = 'exercise01.sqlite'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

Next, we need to do some exploration of the SQLite database to help us understand how to create our flat file.  We can use the code below to figure out how our database has been set up.

In [52]:
#This Code Tells me the Column Names in the SQLite Master Table
c.execute("SELECT * FROM sqlite_master;")
column_names = [description[0] for description in c.description]
print column_names

['type', 'name', 'tbl_name', 'rootpage', 'sql']


Using the above output, we can write a query that shows how each of the database tables were created.  This way, we will be able to figure out how the data needs to be joined to eventually create our flat file.

In [61]:
#Prints the sql column shown above
c.execute("SELECT tbl_name, sql FROM sqlite_master where type = 'table';")
for object in c.fetchall():
    print 'Table Name: ' + object[0]
    print object[1]

Table Name: workclasses
CREATE TABLE workclasses (
	id INTEGER NOT NULL, name TEXT, 
	PRIMARY KEY (id)
)
Table Name: education_levels
CREATE TABLE education_levels (
	id INTEGER NOT NULL, name TEXT, 
	PRIMARY KEY (id)
)
Table Name: marital_statuses
CREATE TABLE marital_statuses (
	id INTEGER NOT NULL, name TEXT, 
	PRIMARY KEY (id)
)
Table Name: occupations
CREATE TABLE occupations (
	id INTEGER NOT NULL, name TEXT, 
	PRIMARY KEY (id)
)
Table Name: relationships
CREATE TABLE relationships (
	id INTEGER NOT NULL, name TEXT, 
	PRIMARY KEY (id)
)
Table Name: races
CREATE TABLE races (
	id INTEGER NOT NULL, name TEXT, 
	PRIMARY KEY (id)
)
Table Name: sexes
CREATE TABLE sexes (
	id INTEGER NOT NULL, name TEXT, 
	PRIMARY KEY (id)
)
Table Name: countries
CREATE TABLE countries (
	id INTEGER NOT NULL, name TEXT, 
	PRIMARY KEY (id)
)
Table Name: records
CREATE TABLE records (
	id INTEGER NOT NULL, age INTEGER, workclass_id INTEGER, education_level_id INTEGER, education_num INTEGER, marital_statu

It looks like the 'records' table should be our starting point, so let's read it into Pandas.

In [99]:
#Read 'records' database table into a Pandas DataFrame
import pandas as pd
records = pd.read_sql_query("SELECT * FROM records;", conn, index_col='id')
records.head()

,age,workclass_id,education_level_id,education_num,marital_status_id,occupation_id,relationship_id,race_id,sex_id,capital_gain,capital_loss,hours_week,country_id,over_50k
id,,,,,,,,,,,,,,
1,39,8,10,13,5,2,2,5,2,2174,0,40,40,0
2,50,7,10,13,3,5,1,5,2,0,0,13,40,0
3,38,5,12,9,1,7,2,5,2,0,0,40,40,0
4,53,5,2,7,3,7,1,3,2,0,0,40,40,0
5,28,5,10,13,3,11,6,3,1,0,0,40,6,0


Now let's talk about joins. The columns ending in **"_id"** above are foreign keys that correspond to other tables in the database that contain the values associated with that categorical variable.  Below is an example for the column **"workclass_id."**

In [126]:
#Examine structure of 'workclasses' database table
workclasses = pd.read_sql_query("SELECT * FROM workclasses;", conn, index_col='id')
workclasses.head()

,name
id,
1,?
2,Federal-gov
3,Local-gov
4,Never-worked
5,Private


So, now that we know how that works.  Here is the code to create our master flat file, applying this same idealogy to all of the tables in our DB.

In [122]:
#Create "flattened" table
records_joined = pd.read_sql_query("""
    SELECT records.id,
        records.age,
        workclasses.name as workclass, 
        education_levels.name as education_level, 
        education_num, 
        marital_statuses.name as marital_status,
        occupations.name as occupation,
        relationships.name as relationship,
        races.name as race,
        sexes.name as sex,
        countries.name as country,
        records.capital_gain,
        records.capital_loss,
        records.hours_week,
        records.over_50k
    FROM records 
    JOIN workclasses ON records.workclass_id = workclasses.id
    JOIN education_levels ON records.education_level_id = education_levels.id
    JOIN marital_statuses ON records.marital_status_id = marital_statuses.id
    JOIN occupations ON records.occupation_id = occupations.id
    JOIN relationships ON records.relationship_id = relationships.id
    JOIN races ON records.race_id = races.id
    JOIN sexes ON records.sex_id = sexes.id
    JOIN countries ON records.country_id = countries.id;""", conn, index_col='id')
records_joined.head()

,age,workclass,education_level,education_num,marital_status,occupation,relationship,race,sex,country,capital_gain,capital_loss,hours_week,over_50k
id,,,,,,,,,,,,,,
1,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,2174,0,40,0
2,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0,0,13,0
3,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,0,0,40,0
4,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,0,0,40,0
5,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,0,0,40,0


###Tasks 2 & 3:
**Export the "flattened" table to a CSV file and import the "flattened" table (or CSV file) into your open source analytic environment of choice (R, Python, Java, etc.) and stage it for analysis.**

Well, since we worked with the SQLite API in Python, we don't actually have to do these steps because we already have our data ready to go.  Go us!  But if we *did* need to write this file to a CSV, or read this file from a CSV, we could do that as follows.

In [128]:
#When uncommented, this code writes the DataFrame to csv in the directory
#records_joined.to_csv('records_joined.csv')

#When uncommented, this code reads the csv file to a DataFrame
#records_joined = pd.read_csv('records_joined.csv', index_col='id')

###Task 4:
**Perform some simple exploratory analysis and generate summary statistics to get a sense of what is in the data.**

In [132]:
#Descriptive Statistics of Continuous Variables
records_joined.describe()

,age,education_num,capital_gain,capital_loss,hours_week,over_50k
count,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,10.078089,1079.067626,87.502314,40.422382,0.239282
std,13.710510,2.570973,7452.019058,403.004552,12.391444,0.426649
min,17.000000,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,16.000000,99999.000000,4356.000000,99.000000,1.000000


In [ ]:
conn.close